In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
#Note: Two ways to write the same thing:

# women = train_data.loc[train_data.Sex == 'female']["Survived"]
women = train_data.loc[train_data["Sex"] == "female","Survived"]


rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


# Encode Sex

In [6]:
train_data.loc[train_data["Sex"] == "male", "Sex"] = 0
train_data.loc[train_data["Sex"] == "female", "Sex"] = 1

print(train_data.shape)
train_data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [7]:
test_data.loc[test_data["Sex"] == "male", "Sex"] = 0
test_data.loc[test_data["Sex"] == "female", "Sex"] = 1


print(test_data.shape)
test_data.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S


# Filling Missing Fares

In [8]:
mean_fare = train_data['Fare'].mean()

test_data.loc[pd.isnull(test_data['Fare']),"Fare"] = mean_fare

#Check for null values --> should return an empty df

test_data[pd.isnull(test_data["Fare"])]


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


# Encode Embarked

In [9]:
embarked = pd.get_dummies(train_data["Embarked"])

train_data = pd.concat([train_data,embarked], axis=1)

#Note: check why bottom line of code doesn't work
train_data.drop(['Embarked'], axis=1)

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,0,0,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,0,0,1


In [10]:
embarked = pd.get_dummies(test_data["Embarked"])

test_data = pd.concat([test_data,embarked], axis=1)

test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S,0,0,1
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q,0,1,0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S,0,0,1


# Encode Age

In [11]:
train_data["Age_Young"] = train_data["Age"] < 15
train_data["Age_Medium"] = (train_data["Age"] >= 15) & (train_data["Age"] < 35)
train_data["Age_Old"] = train_data["Age"] >= 35

print(train_data.shape)
train_data[["Age", "Age_Young", "Age_Medium", "Age_Old"]].head(10)

(891, 18)


,Age,Age_Young,Age_Medium,Age_Old
0,22.0,False,True,False
1,38.0,False,False,True
2,26.0,False,True,False
3,35.0,False,False,True
4,35.0,False,False,True
5,NaN,False,False,False
6,54.0,False,False,True
7,2.0,True,False,False
8,27.0,False,True,False
9,14.0,True,False,False


In [12]:
test_data["Age_Young"] = test_data["Age"] < 15
test_data["Age_Medium"] = (test_data["Age"] >= 15) & (test_data["Age"] < 35)
test_data["Age_Old"] = test_data["Age"] >= 35

print(test_data.shape)
test_data[["Age", "Age_Young", "Age_Medium", "Age_Old"]].head(10)

(418, 17)


,Age,Age_Young,Age_Medium,Age_Old
0,34.5,False,True,False
1,47.0,False,False,True
2,62.0,False,False,True
3,27.0,False,True,False
4,22.0,False,True,False
5,14.0,True,False,False
6,30.0,False,True,False
7,26.0,False,True,False
8,18.0,False,True,False
9,21.0,False,True,False


# Encode Title

In [13]:
title = train_data['Name'].apply(lambda x:x.split(',')[1].split('.')[0])

train_data['Title'] = title.str.strip()

train_data['Title']

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886     Rev
887    Miss
888    Miss
889      Mr
890      Mr
Name: Title, Length: 891, dtype: object

In [14]:
title = test_data['Name'].apply(lambda x:x.split(',')[1].split('.')[0])

test_data['Title'] = title.str.strip()

test_data['Title']

0          Mr
1         Mrs
2          Mr
3          Mr
4         Mrs
        ...  
413        Mr
414      Dona
415        Mr
416        Mr
417    Master
Name: Title, Length: 418, dtype: object

In [15]:
#To view all Titles

train_data.groupby(['Title','Survived'])['Name'].count().sort_values(ascending = False)

#Note^:Name can be replaced can by PassengerId, etc. as it is just counting the number of time it appears for each title.

Title         Survived
Mr            0           436
Miss          1           127
Mrs           1            99
Mr            1            81
Miss          0            55
Mrs           0            26
Master        1            23
              0            17
Rev           0             6
Dr            0             4
              1             3
Mlle          1             2
Jonkheer      0             1
Col           0             1
              1             1
Don           0             1
the Countess  1             1
Lady          1             1
Major         0             1
              1             1
Sir           1             1
Mme           1             1
Ms            1             1
Capt          0             1
Name: Name, dtype: int64

*For Training Data*

In [16]:
#Categorizing rare occurring titles as one
Rare = ['Lady','the Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer']

#Replacing rare and French titles to enlish ones
train_data['Title'] = train_data['Title'].replace(Rare, 'Rare')
train_data['Title'] = train_data['Title'].replace('Mlle', 'Miss')
train_data['Title'] = train_data['Title'].replace('Ms', 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')

#NOTE: see why one results in a dataframe and the other doesn't.

train_data.groupby('Title',as_index = False)['Survived'].mean().sort_values('Survived',ascending = False)


#train_data.groupby('Title')['Survived'].mean().sort_values(ascending = False)

,Title,Survived
3,Mrs,0.793651
1,Miss,0.702703
0,Master,0.575000
4,Rare,0.347826
2,Mr,0.156673


In [17]:
#Converting titles to integers

title_map = {"Miss": 0, "Mrs": 1, "Mr":2, "Master": 3, "Rare": 4}

train_data['Title'] = train_data['Title'].map(title_map)

train_data['Title'] = train_data['Title'].fillna(0)
train_data['Title'] = train_data['Title'].astype(int)

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S,Age_Young,Age_Medium,Age_Old,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1,False,True,False,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,1,0,0,False,False,True,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1,False,True,False,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,0,0,1,False,False,True,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,0,0,1,False,False,True,2


*For Testing Data*

In [18]:
#Categorizing rare occurring titles as one
Rare = ['Lady','the Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer']

#Replacing rare and French titles to enlish ones
test_data['Title'] = test_data['Title'].replace(Rare, 'Rare')
test_data['Title'] = test_data['Title'].replace('Mlle', 'Miss')
test_data['Title'] = test_data['Title'].replace('Ms', 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')

In [19]:
#Converting titles to integers

title_map = {"Miss": 0, "Mrs": 1, "Mr":2, "Master": 3, "Rare": 4}

test_data['Title'] = test_data['Title'].map(title_map)

test_data['Title'] = test_data['Title'].fillna(0)
test_data['Title'] = test_data['Title'].astype(int)

test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S,Age_Young,Age_Medium,Age_Old,Title
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q,0,1,0,False,True,False,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S,0,0,1,False,False,True,1
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q,0,1,0,False,False,True,2
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S,0,0,1,False,True,False,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S,0,0,1,False,True,False,1


# Training the Model

In [20]:
features = ['Pclass','Sex','Fare','C','Q','S','Age_Young','Age_Medium','Age_Old','Title']

X_train = train_data[features]

print(X_train.shape)
X_train.head()

(891, 10)


,Pclass,Sex,Fare,C,Q,S,Age_Young,Age_Medium,Age_Old,Title
0,3,0,7.2500,0,0,1,False,True,False,2
1,1,1,71.2833,1,0,0,False,False,True,1
2,3,1,7.9250,0,0,1,False,True,False,0
3,1,1,53.1000,0,0,1,False,False,True,1
4,3,0,8.0500,0,0,1,False,False,True,2


In [21]:
label = "Survived"

y_train = train_data[label]

print(y_train.shape)
y_train.head()

(891,)


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

# Defining Model

In [22]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 10, random_state = 1)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=1)

# Predict

In [23]:
X_test = test_data[features]

print(X_test.shape)
X_test.head()

(418, 10)


,Pclass,Sex,Fare,C,Q,S,Age_Young,Age_Medium,Age_Old,Title
0,3,0,7.8292,0,1,0,False,True,False,2
1,3,1,7.0000,0,0,1,False,False,True,1
2,2,0,9.6875,0,1,0,False,False,True,2
3,3,0,8.6625,0,0,1,False,True,False,2
4,3,1,12.2875,0,0,1,False,True,False,1


In [24]:
prediction = model.predict(X_test)

print(prediction.shape)
prediction[:20]

(418,)


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0])

# Submission

In [25]:
#output = pd.DataFrame({'Survived': prediction})

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction})
output.to_csv('my_submission_6.csv', index=False)
print("Your submission was successfully saved!")
output

Your submission was successfully saved!


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
